In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers

In [2]:
df = pd.read_csv("../data/clean_EventsAdapt_SentenceSet.csv")

In [3]:
from transformers import GPT2Tokenizer, GPT2Model

In [4]:
torch._C._cuda_getDeviceCount()

1

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"  # Replace with any HF chat model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, return_dict_in_generate=True, output_scores=True, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

/home/areebkhalfay/Repositories/psych-research/lib/python3.12/site-packages/accelerate/utils/modeling.py:1536: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
prompt = df.iloc[0]['Sentence'] + " Is this sentence plausible? Answer only with 1 for yes or 0 for no"
input_ids = tokenizer(prompt, return_tensors="pt").to('cuda').input_ids
output = model.generate(input_ids)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [39]:
generated_ids = output.sequences[0]
response_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
print("Response:", response_text)

Response: The raider caught the illness. Is this sentence plausible? Answer only with 1 for yes or 0 for no.

The raider caught the illness. Is this sentence plausible? Answer only with 1 for


In [20]:
transition_scores = model.compute_transition_scores(
    output.sequences, output.scores, normalize_logits=True
)

In [25]:
log_probs = torch.log_softmax(torch.stack(output.scores), dim=-1)
generated_tokens = generated_ids[input_ids.shape[1]:]

In [26]:
token_log_probs = [log_probs[i, :, token_id].item() for i, token_id in enumerate(generated_tokens)]
print("Log Probabilities:", token_log_probs)

Log Probabilities: [-1.745070457458496, -0.0014392504235729575, -2.4316742420196533, -0.8991115093231201, -0.4054230749607086, -0.10869777202606201, -0.1516532301902771, -0.032304685562849045, -1.0999172925949097, -2.0126843452453613, -0.00022897482267580926, -1.2179912328720093, -0.23104265332221985, -0.34323638677597046, -0.05943372845649719, -0.024750739336013794, -0.0993984043598175, -0.4383017420768738, -0.4482728838920593, -0.00020418466010596603]


In [27]:
total_log_prob = sum(token_log_probs)
print(f"Total Log Probability: {total_log_prob:.4f}")

Total Log Probability: -11.7508
